<a href="https://colab.research.google.com/github/davidcpage/seqdist/blob/master/notebooks/01_CTC_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# default_exp ctc_simple

# CTC loss simple

> A simplified CTC loss for decoding lattices with only two options stay/move. This can be used for decoding without collapsing of repeats.

In [ ]:
#export
import numpy as np
import cupy as cp
import torch
import torch.nn as nn
from collections import namedtuple
from seqdist.utils import *

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Preliminaries

Generate a test example:

In [ ]:
#export
def generate_sample_inputs(T, N, L_min, L_max, device=device):
    """
    Args:
        T: number of time steps
        N: batch size
        L_min, L_max: bounds on target length
    """
    stay_scores = torch.rand(T, N, L_max, device=device, requires_grad=True)
    move_scores = torch.rand(T, N, L_max-1, device=device, requires_grad=True)
    target_lengths = torch.randint(L_min, L_max+1, (N,), device=device)
    return stay_scores, move_scores, target_lengths

In [ ]:
sample_inputs = stay_scores, move_scores, target_lengths = generate_sample_inputs(T=800, N=64, L_min=330, L_max=500)

## Loss implementations

### 1. Basic pytorch

Here's a straightforward implementation in pytorch in logspace.


In [ ]:
#export
from torch.nn.functional import pad
from seqdist.core import semiring, Log, Max

def logZ_fwd(stay_scores, move_scores, target_lengths, S=Log):
    T, N, L = stay_scores.shape
    alpha_0 = stay_scores.new_full((N, L), S.zero); alpha_0[:, 0] = S.one
    beta_T = stay_scores.new_full((N, L), S.zero); beta_T[torch.arange(N), target_lengths - 1] = S.one
    move_scores = pad(move_scores, (1, 0), value=S.zero)
    a = pad(alpha_0, (1, 0), value=S.zero)
    for t in range(0, stay_scores.size(0)):
        a[:, 1:] = S.sum(torch.stack([
            S.mul(stay_scores[t], a[:, 1:]),
            S.mul(move_scores[t], a[:, :-1])
        ]), dim=0)    
    return S.sum(S.mul(a[:, 1:], beta_T), dim=1)

In [ ]:
res = logZ_fwd(*sample_inputs)
res

tensor([974.2330, 984.3713, 984.7618, 983.5563, 977.9988, 980.5652, 982.5593,
        971.0525, 959.5686, 981.7110, 982.6719, 981.7672, 960.4211, 958.2485,
        970.2556, 958.7040, 980.2366, 959.4419, 982.2386, 981.1763, 980.1992,
        977.2121, 960.9742, 982.7201, 980.7336, 983.1967, 981.9204, 983.6624,
        984.1556, 984.8323, 976.8654, 966.7563, 968.9673, 970.4185, 980.5213,
        981.7200, 976.7703, 976.3128, 974.6606, 980.0289, 972.0240, 978.2053,
        971.4316, 969.5367, 969.0744, 975.5978, 968.7987, 973.7723, 971.9407,
        984.4995, 983.2872, 973.0983, 974.6730, 962.9532, 979.3806, 984.2042,
        975.2693, 972.7799, 968.5809, 981.6971, 978.7980, 982.7258, 980.3069,
        978.9800], device='cuda:0', grad_fn=<LogsumexpBackward>)

In [ ]:
#report(benchmark_fwd_bwd((lambda *x: logZ_fwd(*x).sum()), *sample_inputs))

### 2. Pytorch with grad

In [ ]:
#export
def _simple_lattice_fwd_bwd(alpha, beta_T, beta_stay, beta_move, stay_scores, move_scores, S=Log):
    T = alpha.size(0) - 1
    move_scores = pad(move_scores, (1, 1), value=S.zero)
    a = pad(alpha[0], (1, 0), value=S.zero)
    for t in range(0, T):
        a[:, 1:] = S.sum(torch.stack([
            S.mul(stay_scores[t], a[:, 1:]),
            S.mul(move_scores[t, :, :-1], a[:, :-1])
        ]), dim=0)
        alpha[t+1] = a[:, 1:]
    
    b = pad(beta_T, (0, 1), value=S.zero)
    for t in range(T, 0, -1):
        beta_stay[t-1] = S.mul(b[:, :-1], stay_scores[t - 1])
        beta_move[t-1] = S.mul(b[:, 1:], move_scores[t - 1, :, 1:])
        b[:, :-1] = S.sum(torch.stack([beta_stay[t-1], beta_move[t-1]]), dim=0)

def dot(x, y, S=Log, dim=-1):
    return S.sum(S.mul(x, y), dim=dim) 

class LogZ(torch.autograd.Function):
    @staticmethod
    def forward(ctx, stay_scores, move_scores, target_lengths, fwd_bwd_impl, S:semiring):
        T, N, L = stay_scores.shape
        
        alpha = stay_scores.new_full((T + 1, N, L), S.zero) 
        alpha[0, :, 0] = S.one 
    
        beta_stay = stay_scores.new_full((T, N, L), S.zero)
        beta_move = stay_scores.new_full((T, N, L), S.zero)
        beta_T = stay_scores.new_full((N, L), S.zero) 
        beta_T[torch.arange(N), target_lengths - 1] = S.one
        
        fwd_bwd_impl(alpha, beta_T, beta_stay, beta_move, stay_scores, move_scores, S) 
        
        g = S.dsum(torch.cat([S.mul(alpha[:-1], beta_stay), S.mul(alpha[:-1], beta_move)], dim=2), dim=2)
        
        ctx.save_for_backward(g.reshape(T, N, 2, L))
        return dot(alpha[-1], beta_T, S)

    @staticmethod
    def backward(ctx, grad):
        g = ctx.saved_tensors[0] * grad[None, :, None, None]
        return g[:, :, 0], g[:, :, 1, :-1], None, None, None

def logZ_py(stay_scores, move_scores, target_lengths):
    return LogZ.apply(stay_scores, move_scores, target_lengths, _simple_lattice_fwd_bwd, Log)

In [ ]:
#export
mean = lambda f: (lambda *xs: f(*xs).mean())

In [ ]:
fwds, bwds = compare_fwd_bwd(float64(mean(logZ_fwd)), float64(mean(logZ_py)), *sample_inputs)

fwd diff: 0.00e+00
bwd diff: 1.16e-10


### 3. Cupy

NB: we defined beta_move to have size (T, N, L) not the more natural (T, N, L - 1) above. We did this so that we can stack it with beta_stay.

In [ ]:
%%writefile seqdist/cuda/ctc_simple.cu
__device__ __forceinline__ FLOAT max2(FLOAT a, FLOAT a1) {
    return a > a1 ? a : a1; 
}

__device__ __forceinline__ FLOAT logsumexp2(FLOAT a, FLOAT a1) {
    FLOAT maxa = max2(a, a1); 
    return maxa + log(exp(a-maxa) + exp(a1-maxa));
}

__device__ __forceinline__ FLOAT add(FLOAT a, FLOAT b) {return a + b;}
__device__ __forceinline__ FLOAT mul(FLOAT a, FLOAT b) {return a * b;}

extern "C" __global__ void fwd_bwd_logspace(
    FLOAT* __restrict__ alpha, FLOAT* __restrict__ beta_T,
    FLOAT* __restrict__ beta_stay, FLOAT* __restrict__ beta_move, 
    const FLOAT* __restrict__ stay_scores, const FLOAT* __restrict__ move_scores,
    int T, int N, int L
) {
    int bx = blockIdx.x, tx = threadIdx.x;
    if (tx >= L) return;
    extern __shared__ FLOAT smem[];
    if (blockIdx.y == 0) {
        FLOAT a = ZERO, a1 = ZERO;
        a = alpha[bx * L + tx];
        for (int t = 0; t < T; t++) {
            FLOAT *buf = smem + (t % 2) * blockDim.x;
            buf[tx] = a; __syncthreads(); 
            if (tx > 0) {a1 = MUL(move_scores[(t * N + bx) * (L - 1) + tx - 1], buf[tx - 1]);}
            a = SUM(MUL(stay_scores[(t * N + bx) * L + tx], a), a1);
            alpha[((t + 1) * N + bx) * L + tx] = a;
        }
    }
    else {
        FLOAT b = ZERO, b1 = ZERO;
        b = beta_T[bx * L + tx];
        for (int t = T; t > 0; t--) {
            FLOAT *buf = smem + (t % 2) * blockDim.x;
            buf[tx] = b; __syncthreads();
            if (tx < L - 1) {
                b1 = MUL(buf[tx + 1], move_scores[(((t - 1) * N + bx) * (L - 1)) + tx]);
                beta_move[((t - 1) * N + bx) * L + tx] = b1;
            }
            b = MUL(b, stay_scores[(((t - 1) * N + bx) * L) + tx]);
            beta_stay[((t - 1) * N + bx) * L + tx] = b;
            b = SUM(b, b1);
        }
    }
  }

extern "C" __global__ void fwd_bwd_logspace_loop(
    FLOAT* __restrict__ alpha, FLOAT* __restrict__ beta,
    FLOAT* __restrict__ beta_stay, FLOAT* __restrict__ beta_move, 
    const FLOAT* __restrict__ stay_scores, const FLOAT* __restrict__ move_scores,
    int T, int N, int L
) {
    int bx = blockIdx.x, tx = threadIdx.x;
    if (blockIdx.y == 0) {
        FLOAT a;
        for (int t = 0; t < T; t++) {
            for (int j = tx; j < L; j += blockDim.x) {
                a = (j > 0) ? MUL(move_scores[(t * N + bx) * (L - 1) + j - 1], alpha[(t * N + bx) * L + j - 1]) : ZERO;
                alpha[((t + 1) * N + bx) * L + j] = SUM(MUL(stay_scores[(t * N + bx) * L + j], alpha[(t * N + bx) * L + j]), a);
            }
            __syncthreads();
        }
    }
    else {
        FLOAT b, b1;
        for (int t = T; t > 0; t--) {
            for (int j = L - blockDim.x + tx; j >= 0; j -= blockDim.x) {
                b1 = ZERO;
                if (j < L - 1) {
                    b1 = MUL(beta[(t * N + bx) * L + j + 1], move_scores[(((t - 1) * N + bx) * (L - 1)) + j]);
                    beta_move[((t - 1) * N + bx) * L + j] = b1;
                }
                b = MUL(beta[(t * N + bx) * L + j], stay_scores[(((t - 1) * N + bx) * L) + j]);
                beta_stay[((t - 1) * N + bx) * L + j] = b;
                beta[((t - 1) * N + bx) * L + j] = SUM(b, b1);
            }
            __syncthreads();
        }
    }
  }

In [ ]:
#export
from seqdist.utils import *
import cupy as cp

cupy_funcs = {
    (torch.float32, Log): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace', FLOAT='float',  SUM='logsumexp2', MUL='add', ZERO='{:E}'.format(Log.zero)),
    (torch.float64, Log): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace', FLOAT='double', SUM='logsumexp2', MUL='add', ZERO='{:E}'.format(Log.zero)),
    (torch.float32, Max): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace', FLOAT='float',  SUM='max2', MUL='add', ZERO='{:E}'.format(Max.zero)),
    (torch.float64, Max): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace', FLOAT='double', SUM='max2', MUL='add', ZERO='{:E}'.format(Max.zero)),
}

cupy_funcs_loop = {
    (torch.float32, Log): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace_loop', FLOAT='float',  SUM='logsumexp2', MUL='add', ZERO='{:E}'.format(Log.zero)),
    (torch.float64, Log): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace_loop', FLOAT='double', SUM='logsumexp2', MUL='add', ZERO='{:E}'.format(Log.zero)),
    (torch.float32, Max): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace_loop', FLOAT='float',  SUM='max2', MUL='add', ZERO='{:E}'.format(Max.zero)),
    (torch.float64, Max): load_cupy_func('cuda/ctc_simple.cu', 'fwd_bwd_logspace_loop', FLOAT='double', SUM='max2', MUL='add', ZERO='{:E}'.format(Max.zero)),
}

def _simple_lattice_fwd_bwd_cupy(alpha, beta_T, beta_stay, beta_move, stay_scores, move_scores, S:semiring):
    T, N, L = stay_scores.shape
    if L > 1024: #exceeds max threads per block
        return _simple_lattice_fwd_bwd_cupy_loop(alpha, beta_T, beta_stay, beta_move, stay_scores, move_scores, S)
    _bytes = 8 if (stay_scores.dtype == torch.float64) else 4
    with cp.cuda.Device(stay_scores.device.index):
        cupy_funcs[(stay_scores.dtype, S)](grid=(N, 2, 1), block=(L, 1, 1), shared_mem=2*_bytes*L,
               args=(alpha.data_ptr(), beta_T.data_ptr(), beta_stay.data_ptr(), beta_move.data_ptr(),
                     stay_scores.data_ptr(), move_scores.data_ptr(), T, N, L))

def _simple_lattice_fwd_bwd_cupy_loop(alpha, beta_T, beta_stay, beta_move, stay_scores, move_scores, S:semiring, max_block_size=1024):
    T, N, L = stay_scores.shape
    block_size = min(L, max_block_size)
    beta = alpha.new_full(alpha.shape, S.zero)
    beta[-1] = beta_T
    with cp.cuda.Device(stay_scores.device.index):
        cupy_funcs_loop[(stay_scores.dtype, S)](grid=(N, 2, 1), block=(block_size, 1, 1),
               args=(alpha.data_ptr(), beta.data_ptr(), beta_stay.data_ptr(), beta_move.data_ptr(),
                     stay_scores.data_ptr(), move_scores.data_ptr(), T, N, L))
        
def logZ_cupy(stay_scores, move_scores, target_lengths, S:semiring=Log):
    return LogZ.apply(stay_scores, move_scores, target_lengths, _simple_lattice_fwd_bwd_cupy, S)

def viterbi_alignments(stay_scores, move_scores, target_lengths):
    target_lengths = target_lengths.to(stay_scores.device)
    stay_scores, move_scores = stay_scores.detach().requires_grad_(), move_scores.detach().requires_grad_()
    logZ_cupy(stay_scores, move_scores, target_lengths, Max).sum().backward()
    alignments = stay_scores.grad.clone()
    alignments[:, :, :-1] += move_scores.grad
    return alignments

In [ ]:
fwds, bwds = compare_fwd_bwd(float64(mean(logZ_py)), float64(mean(logZ_cupy)), *sample_inputs)

fwd diff: 0.00e+00
bwd diff: 0.00e+00


In [ ]:
report(benchmark_fwd_bwd(mean(logZ_cupy), *sample_inputs, nloops=100))

fwd: 19.56ms (19.31-21.06ms)
bwd: 7.53ms (7.40-8.05ms)
tot: 27.10ms (26.81-29.11ms)
